## Bước 1: Import thư viện cần thiết

Đầu tiên, chúng ta cần import các thư viện. Mình sẽ giải thích từng cái nhé!

In [1]:
# Thư viện cơ bản
import pandas as pd
import numpy as np   # Tính toán số học
import matplotlib.pyplot as plt  # Vẽ biểu đồ
import seaborn as sns

# Thư viện Machine Learning
from sklearn.ensemble import RandomForestClassifier  # Model Random Forest
from sklearn.model_selection import train_test_split  # Chia train/test

# Tắt warning để màn hình sạch sẽ hơn
import warnings
warnings.filterwarnings('ignore')

print("✅ Import thành công!")

✅ Import thành công!


## Bước 2: Load dữ liệu Titanic

Giờ chúng ta sẽ tải dữ liệu Titanic. Dataset này có sẵn trong thư viện seaborn!

In [2]:
# Load dữ liệu Titanic từ seaborn
df = sns.load_dataset('titanic')

print(f"📊 Dataset có {df.shape[0]} hàng và {df.shape[1]} cột")
print("\n🔍 5 dòng đầu tiên:")
df.head()

📊 Dataset có 891 hàng và 15 cột

🔍 5 dòng đầu tiên:


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


### Giải thích các cột quan trọng:
- **survived**: 1 = sống sót, 0 = tử nạn (đây là target chúng ta cần dự đoán)
- **pclass**: Hạng vé (1, 2, 3)
- **sex**: Giới tính
- **age**: Tuổi
- **fare**: Giá vé
- **embarked**: Cảng xuất phát (C, Q, S)

## Bước 3: Chuẩn bị dữ liệu

Trước khi train model, chúng ta cần:
1. Chọn features (đặc trưng) để dùng
2. Xử lý missing values (giá trị bị thiếu)
3. Convert text sang số (vì ML chỉ hiểu số!)

In [4]:
# Chọn các features đơn giản
df_clean = df[['survived', 'pclass', 'sex', 'age', 'fare', 'embarked']].copy()

# Xem có bao nhiêu giá trị bị thiếu
print("🔍 Số giá trị bị thiếu mỗi cột:")
print(df_clean.isnull().sum())

🔍 Số giá trị bị thiếu mỗi cột:
survived      0
pclass        0
sex           0
age         177
fare          0
embarked      2
dtype: int64


In [5]:
# Xử lý missing values đơn giản
df_clean['age'].fillna(df_clean['age'].median(), inplace=True)  # Điền tuổi = median
df_clean['fare'].fillna(df_clean['fare'].median(), inplace=True)  # Điền giá vé = median
df_clean['embarked'].fillna('S', inplace=True)  # Điền cảng phổ biến nhất

# Convert text sang số
df_clean['sex'] = df_clean['sex'].map({'male': 0, 'female': 1})  # male=0, female=1
df_clean['embarked'] = df_clean['embarked'].map({'S': 0, 'C': 1, 'Q': 2})  # S=0, C=1, Q=2

print("✅ Xử lý xong! Kiểm tra lại:")
print(df_clean.isnull().sum())
print("\n📊 Dữ liệu sau khi xử lý:")
df_clean.head()

✅ Xử lý xong! Kiểm tra lại:
survived    0
pclass      0
sex         0
age         0
fare        0
embarked    0
dtype: int64

📊 Dữ liệu sau khi xử lý:


,survived,pclass,sex,age,fare,embarked
0,0,3,0,22.0,7.2500,0
1,1,1,1,38.0,71.2833,1
2,1,3,1,26.0,7.9250,0
3,1,1,1,35.0,53.1000,0
4,0,3,0,35.0,8.0500,0


In [6]:
# Tách X (features) và y (target)
X = df_clean.drop('survived', axis=1)  # X = tất cả cột trừ 'survived'
y = df_clean['survived']  # y = cột 'survived'

print(f"✅ X shape: {X.shape}")
print(f"✅ y shape: {y.shape}")

✅ X shape: (891, 5)
✅ y shape: (891,)


In [ ]:
# Chia train/test (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"📚 Train set: {X_train.shape[0]} samples")
print(f"📝 Test set: {X_test.shape[0]} samples")

---
# Bài 1: So sánh Grid Search vs Random Search

## Vấn đề: "Lời nguyền số chiều" là gì?

Khi chúng ta có **nhiều hyperparameter** cần tune, số lượng tổ hợp tăng **CỰC NHANH**!

Ví dụ:
- 3 tham số, mỗi cái 10 giá trị → 10 × 10 × 10 = **1,000 tổ hợp**
- 5 tham số, mỗi cái 10 giá trị → 10^5 = **100,000 tổ hợp**
- 10 tham số, mỗi cái 10 giá trị → 10^10 = **10 tỷ tổ hợp** 😱

→ Đây gọi là **"Curse of Dimensionality"** (Lời nguyền số chiều)

## Phần A: Tính số tổ hợp Grid Search

Giờ mình sẽ định nghĩa một không gian hyperparameter và tính xem phải thử bao nhiêu tổ hợp!

In [ ]:
# Định nghĩa không gian hyperparameter
param_grid = {
    'n_estimators': [50, 100, 200],        # Số cây trong rừng
    'max_depth': [5, 10, 15, 20],          # Độ sâu tối đa của cây
    'min_samples_split': [2, 5, 10],       # Số mẫu tối thiểu để split node
    'min_samples_leaf': [1, 2, 4],         # Số mẫu tối thiểu ở lá
    'max_features': ['sqrt', 'log2']       # Số features xét mỗi split
}

print("📝 Không gian hyperparameter:")
print("="*50)

total = 1
for param, values in param_grid.items():
    n = len(values)
    print(f"{param:20s}: {n} giá trị {values}")
    total *= n

print("="*50)
print(f"\n🔢 TỔNG SỐ TỔ HỢP: {total}")
print(f"\n⏰ Nếu dùng 5-fold Cross-Validation:")
print(f"   → Phải train model: {total} × 5 = {total * 5} lần!")
print(f"   → Giả sử 1 lần train mất 2 giây")
print(f"   → Tổng thời gian: {total * 5 * 2 / 60:.1f} phút")

### 💡 Giải thích:

- Grid Search phải thử **TẤT CẢ** các tổ hợp
- 3 × 4 × 3 × 3 × 2 = **216 tổ hợp**
- Với 5-fold CV = **1,080 lần training**!
- Đây mới chỉ là 5 hyperparameters, trong thực tế có thể có hơn 10!

→ **Vấn đề**: Tốn cực kỳ nhiều thời gian! ⏰

## Phần B: Random Search - Giải pháp thông minh

**Ý tưởng**: Thay vì thử TẤT CẢ, chỉ thử **NGẪU NHIÊN một số tổ hợp**!

Nghe có vẻ ngớ ngẩn nhưng thực tế **rất hiệu quả**! 🎯

In [ ]:
# Import RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Tạo Random Forest model
rf = RandomForestClassifier(random_state=42)

# Chạy Random Search với CHỈ 30 lần thử
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_grid,
    n_iter=30,  # CHỈ thử 30 tổ hợp ngẫu nhiên!
    cv=5,       # 5-fold Cross-Validation
    scoring='accuracy',
    random_state=42,
    n_jobs=-1,  # Dùng tất cả CPU cores
    verbose=1
)

print("🚀 Bắt đầu Random Search...\n")
random_search.fit(X_train, y_train)
print("\n✅ Hoàn thành!")

In [ ]:
# Xem kết quả
print("="*60)
print("🏆 KẾT QUẢ RANDOM SEARCH")
print("="*60)
print(f"\n📊 Best Score (CV Accuracy): {random_search.best_score_:.4f}")
print(f"\n⚙️  Best Hyperparameters:")
for param, value in random_search.best_params_.items():
    print(f"   {param:20s}: {value}")

# Test trên test set
test_score = random_search.score(X_test, y_test)
print(f"\n✨ Test Accuracy: {test_score:.4f}")
print("="*60)

### 📊 So sánh Grid Search vs Random Search:

| Phương pháp | Số tổ hợp thử | Số lần train (5-fold) | % tài nguyên |
|-------------|---------------|----------------------|-------------|
| **Grid Search** | 216 | 1,080 | 100% |
| **Random Search** | 30 | 150 | **13.9%** |

### ✅ Kết luận:
- Random Search chỉ dùng ~14% tài nguyên!
- Nhưng vẫn tìm được hyperparameter tốt!
- **Tại sao hiệu quả?** Vì không phải tất cả hyperparameters đều quan trọng như nhau!

---
# Bài 2: TPE - Thuật toán "Có Trí Nhớ"

## Vấn đề với Random Search:
- Random Search vẫn chọn ngẫu nhiên
- Không "học" từ các lần thử trước
- Có thể lãng phí tài nguyên vào vùng tham số xấu

## Giải pháp: TPE (Tree-structured Parzen Estimator)
- **Học** từ các lần thử trước
- **Tập trung** vào vùng tham số hứa hẹn
- Thuộc dạng **Bayesian Optimization**

## Cài đặt Optuna

Optuna là thư viện tối ưu hyperparameter rất mạnh, dùng TPE làm thuật toán mặc định!

In [ ]:
# Nếu chưa cài, chạy lệnh này:
# !pip install optuna

import optuna
from optuna.samplers import TPESampler

print("✅ Import Optuna thành công!")
print(f"📦 Optuna version: {optuna.__version__}")

## Định nghĩa Objective Function

Đây là hàm mà Optuna sẽ cố gắng tối ưu!

In [ ]:
from sklearn.model_selection import cross_val_score

def objective(trial):
    """
    Hàm objective cho Optuna
    trial: đối tượng để suggest hyperparameters
    """
    # Suggest hyperparameters (Optuna tự động chọn)
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 4),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        'random_state': 42
    }
    
    # Train model với hyperparameters này
    model = RandomForestClassifier(**params)
    
    # Tính accuracy bằng cross-validation
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    accuracy = scores.mean()
    
    return accuracy  # Return giá trị cần tối ưu

print("✅ Định nghĩa objective function xong!")

## Chạy TPE Optimization!

In [ ]:
# Tạo study với TPE sampler
study = optuna.create_study(
    direction='maximize',  # Maximize accuracy
    sampler=TPESampler(seed=42)  # Dùng TPE algorithm
)

# Chạy optimization với 30 trials (giống Random Search để so sánh)
print("🚀 Bắt đầu TPE Optimization...\n")
study.optimize(objective, n_trials=30, show_progress_bar=True)
print("\n✅ Hoàn thành!")

In [ ]:
# Xem kết quả
print("="*60)
print("🏆 KẾT QUẢ TPE OPTIMIZATION")
print("="*60)
print(f"\n📊 Best Score (CV Accuracy): {study.best_value:.4f}")
print(f"\n⚙️  Best Hyperparameters:")
for param, value in study.best_params.items():
    print(f"   {param:20s}: {value}")
print("="*60)

In [ ]:
# Train model với best params và test
best_model = RandomForestClassifier(**study.best_params, random_state=42)
best_model.fit(X_train, y_train)
test_score_tpe = best_model.score(X_test, y_test)

print(f"✨ Test Accuracy (TPE): {test_score_tpe:.4f}")

## Visualize quá trình tối ưu

Đây là phần thú vị! Chúng ta sẽ xem TPE "học" như thế nào!

In [ ]:
# Vẽ optimization history
from optuna.visualization import plot_optimization_history

fig = plot_optimization_history(study)
fig.update_layout(
    title="TPE Optimization History - Titanic Dataset",
    xaxis_title="Trial Number",
    yaxis_title="Accuracy"
)
fig.show()

print("\n📈 Giải thích:")
print("- Đường xanh: Accuracy của mỗi trial")
print("- Đường đỏ: Best accuracy tìm được cho đến lúc đó")
print("- Nhận xét: Accuracy tăng dần → TPE đang 'học'!")

In [ ]:
# Vẽ param importances
from optuna.visualization import plot_param_importances

fig = plot_param_importances(study)
fig.update_layout(title="Hyperparameter Importance")
fig.show()

print("\n🎯 Giải thích:")
print("- Biểu đồ này cho biết hyperparameter nào QUAN TRỌNG nhất")
print("- TPE tự động phát hiện điều này!")
print("- Giúp chúng ta tập trung vào các tham số quan trọng")

### 🧠 TPE hoạt động như thế nào?

**Ý tưởng chính**:

1. TPE chia các trials thành 2 nhóm:
   - **Nhóm TỐT** (l): Top 20% trials có accuracy cao nhất
   - **Nhóm XẤU** (g): 80% còn lại

2. TPE học 2 phân phối:
   - $l(x)$: Phân phối hyperparameters trong nhóm TỐT
   - $g(x)$: Phân phối hyperparameters trong nhóm XẤU

3. TPE chọn $x$ mới sao cho maximize $\frac{l(x)}{g(x)}$
   - Tức là: Chọn $x$ có xác suất **CAO** trong nhóm TỐT
   - Và xác suất **THẤP** trong nhóm XẤU

→ Đây chính là "trí nhớ" của TPE! 🧠

---
# Bài 3: So sánh Random Search vs TPE

Giờ chúng ta sẽ so sánh trực tiếp để thấy sự khác biệt!

## Chạy Random Search với Optuna

(Để công bằng, dùng cùng framework Optuna)

In [ ]:
from optuna.samplers import RandomSampler

# Tạo study với Random Sampler
study_random = optuna.create_study(
    direction='maximize',
    sampler=RandomSampler(seed=42)  # Dùng Random sampling
)

print("🎲 Chạy Random Search với Optuna...\n")
study_random.optimize(objective, n_trials=30, show_progress_bar=True)
print("\n✅ Hoàn thành!")

In [ ]:
# So sánh kết quả
print("\n" + "="*70)
print("📊 SO SÁNH RANDOM SEARCH VS TPE")
print("="*70)

print(f"\n🎲 Random Search (Vô nhớ):")
print(f"   Best Accuracy: {study_random.best_value:.4f}")

print(f"\n🧠 TPE (Có nhớ):")
print(f"   Best Accuracy: {study.best_value:.4f}")

if study.best_value > study_random.best_value:
    improvement = (study.best_value - study_random.best_value) / study_random.best_value * 100
    print(f"\n✨ TPE TỐT HƠN Random Search: +{improvement:.2f}%")
else:
    print(f"\n⚠️  Lần này Random Search may mắn hơn (nhưng không ổn định)")

print("="*70)

## So sánh tốc độ hội tụ

In [ ]:
# Lấy best value history
random_history = []
tpe_history = []

random_best = 0
for trial in study_random.trials:
    if trial.value > random_best:
        random_best = trial.value
    random_history.append(random_best)

tpe_best = 0
for trial in study.trials:
    if trial.value > tpe_best:
        tpe_best = trial.value
    tpe_history.append(tpe_best)

# Vẽ biểu đồ
plt.figure(figsize=(12, 6))
plt.plot(random_history, label='Random Search', linewidth=2.5, marker='o', markersize=5, alpha=0.7)
plt.plot(tpe_history, label='TPE', linewidth=2.5, marker='s', markersize=5, alpha=0.7)
plt.xlabel('Trial Number', fontsize=12)
plt.ylabel('Best Accuracy Found', fontsize=12)
plt.title('Tốc độ Hội tụ: Random Search vs TPE', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n📊 Giải thích:")
print("- Random Search: Tăng chậm, dao động nhiều")
print("- TPE: Tăng nhanh hơn, ổn định hơn")
print("- TPE 'học' và tập trung vào vùng tốt!")

## Visualize không gian tìm kiếm

Xem cách 2 thuật toán "explore" không gian hyperparameter!

In [ ]:
# Lấy giá trị 2 hyperparameters quan trọng
random_n_est = [t.params['n_estimators'] for t in study_random.trials]
random_depth = [t.params['max_depth'] for t in study_random.trials]
random_scores = [t.value for t in study_random.trials]

tpe_n_est = [t.params['n_estimators'] for t in study.trials]
tpe_depth = [t.params['max_depth'] for t in study.trials]
tpe_scores = [t.value for t in study.trials]

# Vẽ
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Random Search
scatter1 = axes[0].scatter(random_n_est, random_depth, c=random_scores, 
                          s=150, cmap='viridis', edgecolors='black', linewidth=1.5, alpha=0.8)
axes[0].set_xlabel('n_estimators', fontsize=12)
axes[0].set_ylabel('max_depth', fontsize=12)
axes[0].set_title('Random Search\n(Thử ngẫu nhiên)', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)
plt.colorbar(scatter1, ax=axes[0], label='Accuracy')

# TPE
scatter2 = axes[1].scatter(tpe_n_est, tpe_depth, c=tpe_scores, 
                          s=150, cmap='viridis', edgecolors='black', linewidth=1.5, alpha=0.8)
axes[1].set_xlabel('n_estimators', fontsize=12)
axes[1].set_ylabel('max_depth', fontsize=12)
axes[1].set_title('TPE\n(Tập trung vào vùng tốt)', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)
plt.colorbar(scatter2, ax=axes[1], label='Accuracy')

plt.tight_layout()
plt.show()

print("\n🎯 Nhận xét:")
print("- Random Search: Điểm thử phân bố ĐỀU khắp không gian")
print("- TPE: Điểm thử TẬP TRUNG vào vùng có accuracy cao (màu vàng)")
print("- TPE thông minh hơn trong việc sử dụng tài nguyên!")

---
# 📚 Tổng Kết

## 🎯 Bài 1: Curse of Dimensionality

**Vấn đề**: 
- Grid Search phải thử TẤT CẢ tổ hợp → Số lượng tăng MŨ theo số hyperparameters
- Ví dụ: 5 params với ~3 giá trị mỗi cái → ~243 tổ hợp → 1,215 lần training (với 5-fold CV)

**Giải pháp**:
- Random Search chỉ thử một PHẦN ngẫu nhiên (30-50 trials)
- Tiết kiệm ~85% tài nguyên nhưng vẫn hiệu quả!

---

## 🧠 Bài 2: TPE - "Trí nhớ" trong tối ưu

**Ý tưởng**:
- TPE chia trials thành nhóm TỐT ($l$) và XẤU ($g$)
- Học phân phối hyperparameters từ 2 nhóm
- Chọn hyperparameters mới để maximize $\frac{l(x)}{g(x)}$

**Ưu điểm**:
- "Học" từ quá khứ → không lãng phí tài nguyên
- Tự động xác định hyperparameters quan trọng
- Hội tụ nhanh hơn Random Search

---

## ⚖️ Bài 3: So sánh

| Phương pháp | Chiến lược | Ưu điểm | Nhược điểm |
|-------------|-----------|---------|------------|
| **Grid Search** | Thử tất cả | Đầy đủ | Cực kỳ chậm |
| **Random Search** | Thử ngẫu nhiên | Nhanh, đơn giản | Không học từ quá khứ |
| **TPE** | Học & tập trung | Nhanh + Thông minh | Cần setup phức tạp hơn |

---

## 💡 Khuyến nghị thực tế:

1. **Không gian nhỏ** (< 50 tổ hợp): Dùng **Grid Search**
2. **Không gian vừa, cần nhanh**: Dùng **Random Search**
3. **Không gian lớn, muốn tốt nhất**: Dùng **TPE/Optuna** ⭐
4. **Production**: Luôn dùng TPE/Bayesian Optimization!

---

## 🎓 Bài học quan trọng:

1. **Không phải hyperparameter nào cũng quan trọng như nhau**
   → Random Search hiệu quả vì tập trung vào chiều quan trọng

2. **"Memory" (trí nhớ) rất quan trọng**
   → TPE học từ quá khứ để tối ưu tương lai

3. **Trade-off giữa exploration và exploitation**
   - Grid/Random: Exploration nhiều
   - TPE: Balance tốt giữa explore (thử vùng mới) và exploit (khai thác vùng tốt)

---

### 🚀 Thực hành thêm:

- Thử với dataset khác (housing prices, iris, etc.)
- Thử với model khác (SVM, XGBoost, Neural Networks)
- Thử nhiều trials hơn (100, 200) để thấy sự khác biệt rõ hơn
- Tìm hiểu về các sampler khác: CmaEsSampler, NSGAIISampler (multi-objective)

**Chúc bạn học tốt! 🎉**